# Taller 1: Análisis de Actividad Sísmica en Colombia
**Autor:** Torres Anzola Arnold Santiago

Este notebook contiene el proceso completo de limpieza, modelado y evaluación de sismos en Colombia utilizando K-Means Clustering.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import os

# Configuración
RANDOM_STATE = 42
FILE_PATH = 'data/earthquakes.csv'

## 1. Carga y Limpieza de Datos
Cargamos los datos enriquecidos y realizamos un filtrado para asegurar la calidad de los clusters.

In [ ]:
df = pd.read_csv(FILE_PATH)
features = ['latitude', 'longitude', 'depth', 'mag']
X = df[features].copy()
print(f"Registros cargados: {len(df)}")

## 2. Escalado de Datos
Es fundamental escalar los datos para que la profundidad (en km) no domine sobre la magnitud.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 3. Selección de K (Método del Codo)
Evaluamos la inercia para diferentes valores de K.

In [ ]:
inertias = []
K_range = range(2, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(K_range, inertias, 'bo-')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Inercia')
plt.title('Método del Codo')
plt.show()

## 4. Clasificación Final (K=7)
Aplicamos el modelo definitivo con la semilla de reproducibilidad.

In [ ]:
kmeans_final = KMeans(n_clusters=7, random_state=RANDOM_STATE, n_init=10)
df['cluster'] = kmeans_final.fit_predict(X_scaled)
print("Modelado completado con éxito.")

## 5. Perfilado de Clusters
Analizamos qué significa cada zona descubierta.

In [ ]:
cluster_stats = df.groupby('cluster').agg({
    'depth': 'mean',
    'mag': 'mean',
    'latitude': 'count'
}).rename(columns={'latitude': 'count'})
print(cluster_stats)